In [53]:
# Import packages
import pandas as pd
import altair as alt

from vega_datasets import data

In [54]:
# Read in clean data
df = pd.read_csv("../data/clean_data.csv")
df.head()

,Decision,State,GPA,WorkExp,TestScore,WritingScore,Gender,VolunteerLevel
0,Admit,California,3.90,6.7,962.0,100,1.0,0
1,Admit,Florida,3.80,1.4,969.0,97,1.0,4
2,Admit,Colorado,3.60,0.9,969.0,97,0.0,2
3,Admit,California,3.88,1.5,967.0,95,1.0,5
4,Admit,California,3.70,1.2,969.0,94,1.0,2


In [55]:
# Adding per state admission rates
for s in df["State"].unique():
    curr = df[df["State"] == s]
    admit_count = curr[curr["Decision"] == "Admit"].shape[0]
    wait_count = curr[curr["Decision"] == "Waitlist"].shape[0]
    decline_count = curr[curr["Decision"] == "Decline"].shape[0]
    df.loc[df["State"] == s, "AdmitRate"] = (admit_count/curr.shape[0]) * 100
    df.loc[df["State"] == s, "WaitlistRate"] = (wait_count/curr.shape[0]) * 100
    df.loc[df["State"] == s, "DeclineRate"] = (decline_count/curr.shape[0]) * 100
df.head()

,Decision,State,GPA,WorkExp,TestScore,WritingScore,Gender,VolunteerLevel,AdmitRate,WaitlistRate,DeclineRate
0,Admit,California,3.90,6.7,962.0,100,1.0,0,75.000000,16.666667,8.333333
1,Admit,Florida,3.80,1.4,969.0,97,1.0,4,31.428571,31.428571,37.142857
2,Admit,Colorado,3.60,0.9,969.0,97,0.0,2,47.368421,21.052632,31.578947
3,Admit,California,3.88,1.5,967.0,95,1.0,5,75.000000,16.666667,8.333333
4,Admit,California,3.70,1.2,969.0,94,1.0,2,75.000000,16.666667,8.333333


In [69]:
# Get relevant column averages by state
averages_by_state = df.groupby('State')[["AdmitRate", "WaitlistRate", "DeclineRate", "GPA", "TestScore", "WritingScore", "VolunteerLevel"]].mean().reset_index()
state_id_dict = dict(zip(data.population_engineers_hurricanes()["state"], data.population_engineers_hurricanes()["id"]))
# Map state ids
averages_by_state["StateID"] = averages_by_state["State"].map(state_id_dict)
averages_by_state

,State,AdmitRate,WaitlistRate,DeclineRate,GPA,TestScore,WritingScore,VolunteerLevel,StateID
0,Alabama,0.000000,100.000000,0.000000,3.410000,868.000000,85.000000,0.000000,1
1,California,75.000000,16.666667,8.333333,3.600833,916.166667,90.833333,2.666667,6
2,Colorado,47.368421,21.052632,31.578947,3.600526,881.894737,80.684211,2.368421,8
3,Florida,31.428571,31.428571,37.142857,3.559143,872.571429,80.428571,2.457143,12
4,Georgia,0.000000,100.000000,0.000000,3.560000,866.000000,89.000000,1.000000,13
5,Mississippi,0.000000,0.000000,100.000000,2.910000,753.000000,77.000000,1.000000,28
6,New York,0.000000,100.000000,0.000000,3.550000,866.000000,74.000000,1.000000,36
7,Oregon,0.000000,0.000000,100.000000,2.900000,769.000000,87.000000,4.000000,41
8,Utah,16.666667,50.000000,33.333333,3.443333,850.666667,78.333333,2.500000,49
9,Vermont,0.000000,100.000000,0.000000,3.530000,862.000000,85.000000,1.000000,50


In [83]:
states = alt.topo_feature('https://raw.githubusercontent.com/vega/vega-datasets/master/data/us-10m.json', 'states')
click = alt.selection_multi(fields = ["State"])

existing_states = alt.Chart(states).mark_geoshape().encode(
    color = "AdmitRate:Q",
    tooltip = ["State:N", "AdmitRate:Q"],
    opacity = alt.condition('isValid(datum.AdmitRate)', alt.value(1), alt.value(0.2)),
).transform_lookup(
    lookup = "id",
    from_ = alt.LookupData(averages_by_state, "StateID", list(averages_by_state.columns))
).properties(width = 500, height = 300).add_selection(click).project(type = "albersUsa")

missing_states = (
    alt.Chart(states)
    .mark_geoshape(fill = "lightgrey", stroke = "white")
    .encode(opacity=alt.condition("isValid(datum.AdmitRate)", alt.value(0), alt.value(0.2))).add_selection(click).project(type = "albersUsa"))

full_map = existing_states + missing_states

bars = (
    alt.Chart(
        averages_by_state.nlargest(50, "AdmitRate"),
        title = "Top States by Admission Rate").mark_bar().encode(
    x = "AdmitRate",
    opacity = alt.condition(click, alt.value(1), alt.value(0.2)),
    color = "AdmitRate",
    y = alt.Y("State", sort = "x"))
.add_selection(click))

full_map & bars

/Users/miamayerhofer/opt/anaconda3/envs/dsan6600/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/miamayerhofer/opt/anaconda3/envs/dsan6600/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/miamayerhofer/opt/anaconda3/envs/dsan6600/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)